# Preparing notebook and folders

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['PYTHONHASHSEED'] = '2'

In [2]:
import warnings
import time

import numpy as np
np.random.seed(18)
import matplotlib.pyplot as plt
from keras.models import load_model

import concurrent.futures
import functools

from F2_func_file import *

/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.1 when it was built against 1.14.0, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [3]:
#Create folder for storing generated data files
if not os.path.exists("Data_files"):
    warnings.warn("""The folder "Data_files" should already exist. Please, run the "Fig_2_Data_generation" notebook first before proceeding with the "Data_evaluation" one.""")
if not os.path.exists("Data_files/Evaluated_data"):
    os.mkdir("Data_files/Evaluated_data")

In [4]:
#Load the DAMN model
DAMN_model = load_model("DAMN_model.h5", custom_objects={"custom_mse_func": custom_mse_func, "custom_mae_func": custom_mae_func})

#Specify the PSF_width value used to generate data for panels (A), (C), and (D); requirement for Richardson-Lucy algorithm
PSF_width_value = 2

In [5]:
#To fasten the Richardson_lucy algorithm, we split the data evaluation among several CPU units using ProcessPoolExecutor from concurrent.futures
CPU_units_to_use = 10   #Set to 1 if you are not familiar with ProcessPoolExecutor or your CPU availability

## Evaluate dataset of the SNR graph - panel (A)

In [6]:
#Loading the data generated in the Data_generation notebook
data_in_SNR = np.load("Data_files/Generated_data/SNR_data_low_res.npy")
data_target_SNR = np.load("Data_files/Generated_data/SNR_data_high_res.npy")

shape_SNR = data_in_SNR.shape
print("Data shapes:", shape_SNR)

Data shapes: (51, 100, 50, 50)


In [7]:
#Adjust the data shapes for easier model evaluation
data_in_SNR_reshaped = np.reshape(data_in_SNR, (shape_SNR[0]*shape_SNR[1], shape_SNR[2], shape_SNR[3], 1))
data_target_SNR_reshaped = np.reshape(data_target_SNR, (shape_SNR[0]*shape_SNR[1], shape_SNR[2], shape_SNR[3], 1))

print("Data reshaped:", data_in_SNR_reshaped.shape)

Data reshaped: (5100, 50, 50, 1)


### DAMN model

In [8]:
#Use the DAMN model to predict high-resolution images
DAMN_model_output_SNR_reshaped = np.squeeze(DAMN_model.predict(data_in_SNR_reshaped))

#And return to the original shape
DAMN_model_output_SNR = DAMN_model_output_SNR_reshaped.reshape((shape_SNR[0], shape_SNR[1], shape_SNR[2], shape_SNR[3]))

np.save("Data_files/Evaluated_data/SNR_DAMN_model_output.npy", DAMN_model_output_SNR)

160/160 [==============================] - 15s 82ms/step


In [9]:
#Evaluate a chosen metric comparing the DAMN output with the corresponding target image
DAMN_model_errors_SNR = Evaluate_metric(DAMN_model_output_SNR, data_target_SNR)

np.save("Data_files/Evaluated_data/SNR_DAMN_model_errors.npy", DAMN_model_errors_SNR)

### Richardson-Lucy algorithm

In [10]:
#As the RL algorithm is very time consuming (matter of up to hours), you can choose to evaluate only a small portion of the data for the graph visualization
#Namely, 1/10 of data samples in every 4-th point on the horizontal axis
use_all_data_SNR = True             #Switch to False for evaluating only the reduced dataset 

if use_all_data_SNR:
    RL_data_in_SNR = data_in_SNR
    RL_data_target_SNR = data_target_SNR
    RL_output_SNR = np.zeros(data_in_SNR.shape)
else:
    RL_data_in_SNR = data_in_SNR[::4,:10]
    RL_data_target_SNR = data_target_SNR[::4,:10]
    RL_output_SNR = np.zeros(data_in_SNR[::4,:10].shape)

print("Using the following data shape for RL algorithm:", RL_data_in_SNR.shape)

Using the following data shape for RL slgorithm: (51, 100, 50, 50)


##### The following cell runs the Richardson-Lucy algorithm, which might turn very time consuming

In [ ]:
#The kernel stays the same for all SNR data samples
kernel = Gauss_kernel(PSF_width_value)

#The ProcessPoolExecutor calling RL_iteration_for_concurrent function to evaluate the RL_data_in_SNR data
start = time.time()
for i in range(RL_data_in_SNR.shape[0]):
    start_i = time.time()
    with concurrent.futures.ProcessPoolExecutor(CPU_units_to_use) as pool:
        intermediate_func = functools.partial(RL_iteration_for_concurrent, kernel)
        res = pool.map(intermediate_func, RL_data_in_SNR[i])
    RL_output_SNR[i] = np.array(list(res))
    print("Finished iteration", i+1, "out of", RL_data_in_SNR.shape[0], "in", np.round(time.time()-start_i, 2), "seconds (" + str(np.round(time.time()-start, 2)), "from start).")

np.save("Data_files/Evaluated_data/SNR_RL_output.npy", RL_output_SNR)

Finished iteration 1 out of 51 in 116.32 seconds (116.33 from start).
Finished iteration 2 out of 51 in 117.26 seconds (233.59 from start).
Finished iteration 3 out of 51 in 117.78 seconds (351.37 from start).
Finished iteration 4 out of 51 in 120.19 seconds (471.55 from start).
Finished iteration 5 out of 51 in 119.48 seconds (591.04 from start).
Finished iteration 6 out of 51 in 121.37 seconds (712.41 from start).
Finished iteration 7 out of 51 in 122.44 seconds (834.85 from start).
Finished iteration 8 out of 51 in 124.21 seconds (959.06 from start).
Finished iteration 9 out of 51 in 129.5 seconds (1088.56 from start).
Finished iteration 10 out of 51 in 130.58 seconds (1219.14 from start).
Finished iteration 11 out of 51 in 132.46 seconds (1351.6 from start).
Finished iteration 12 out of 51 in 136.97 seconds (1488.57 from start).
Finished iteration 13 out of 51 in 141.07 seconds (1629.64 from start).
Finished iteration 14 out of 51 in 144.62 seconds (1774.26 from start).
Finished it

In [ ]:
#Evaluate a chosen metric comparing the DAMN output with the corresponding target image
RL_errors_SNR = Evaluate_metric(RL_output_SNR, RL_data_target_SNR)

np.save("Data_files/Evaluated_data/SNR_RL_errors.npy", RL_errors_SNR)

## Evaluate dataset of the PSF width graph - panel (B)

In [ ]:
#Loading the data generated in the Data_generation notebook
data_in_PSF = np.load("Data_files/Generated_data/PSF_data_low_res.npy")
data_target_PSF = np.load("Data_files/Generated_data/PSF_data_high_res.npy")
horizontal_axis_PSF = np.load("Data_files/Generated_data/PSF_axis_array.npy")

shape_PSF = data_in_PSF.shape
print("Data shapes:", shape_PSF)

In [ ]:
#Adjust the data shapes for easier model evaluation
data_in_PSF_reshaped = np.reshape(data_in_PSF, (shape_PSF[0]*shape_PSF[1], shape_PSF[2], shape_PSF[3], 1))
data_target_PSF_reshaped = np.reshape(data_target_PSF, (shape_PSF[0]*shape_PSF[1], shape_PSF[2], shape_PSF[3], 1))

print("Data reshaped:", data_in_PSF_reshaped.shape)

### DAMN model

In [ ]:
#Use the DAMN model to predict high-resolution images
DAMN_model_output_PSF_reshaped = np.squeeze(DAMN_model.predict(data_in_PSF_reshaped))

#And return to the original shape
DAMN_model_output_PSF = DAMN_model_output_PSF_reshaped.reshape((shape_PSF[0], shape_PSF[1], shape_PSF[2], shape_PSF[3]))

np.save("Data_files/Evaluated_data/PSF_DAMN_model_output.npy", DAMN_model_output_PSF)

In [ ]:
#Evaluate a chosen metric comparing the DAMN output with the corresponding target image
DAMN_model_errors_PSF = Evaluate_metric(DAMN_model_output_PSF, data_target_PSF)

np.save("Data_files/Evaluated_data/PSF_DAMN_model_errors.npy", DAMN_model_errors_PSF)

### Richardson-Lucy algorithm

In [ ]:
#As the RL algorithm is very time consuming (matter of up to hours), you can choose to evaluate only a small portion of the data for the graph visualization
#Namely, 1/10 of data samples in every 4-th point on the horizontal axis
use_all_data_PSF = True             #Switch to False for evaluating only the reduced dataset 

if use_all_data_PSF:
    RL_data_in_PSF = data_in_PSF
    RL_data_target_PSF = data_target_PSF
    RL_horizontal_axis_PSF = horizontal_axis_PSF
    RL_output_PSF = np.zeros(data_in_PSF.shape)
else:
    RL_data_in_PSF = data_in_PSF[::4,:10]
    RL_data_target_PSF = data_target_PSF[::4,:10]
    RL_horizontal_axis_PSF = horizontal_axis_PSF[::4]
    RL_output_PSF = np.zeros(data_in_PSF[::4,:10].shape)

print("Using the following data shape for RL slgorithm:", RL_data_in_PSF.shape)

##### The following cell runs the Richardson-Lucy algorithm, which might turn very time consuming

In [ ]:
#The ProcessPoolExecutor calling RL_iteration_for_concurrent function to evaluate the RL_data_in_PSF data
start = time.time()
for i in range(RL_data_in_PSF.shape[0]):
    start_i = time.time()

    #The kernel stays the same only for data with the same horizontal axis position
    kernel = Gauss_kernel(RL_horizontal_axis_PSF[i])
    with concurrent.futures.ProcessPoolExecutor(CPU_units_to_use) as pool:
        intermediate_func = functools.partial(RL_iteration_for_concurrent, kernel)
        res = pool.map(intermediate_func, RL_data_in_PSF[i])
    RL_output_PSF[i] = np.array(list(res))
    print("Finished iteration", i+1, "out of", RL_data_in_PSF.shape[0], "in", np.round(time.time()-start_i, 2), "seconds (" + str(np.round(time.time()-start, 2)), "from start).")

np.save("Data_files/Evaluated_data/PSF_RL_output.npy", RL_output_PSF)

In [ ]:
#Evaluate a chosen metric comparing the DAMN output with the corresponding target image
RL_errors_PSF = Evaluate_metric(RL_output_PSF, RL_data_target_PSF)

np.save("Data_files/Evaluated_data/PSF_RL_errors.npy", RL_errors_PSF)

## Evaluate dataset of the Emitter Concentration graph - panel (C)

In [ ]:
#Loading the data generated in the Data_generation notebook
data_in_Conc = np.load("Data_files/Generated_data/Concentration_data_low_res.npy")
data_target_Conc = np.load("Data_files/Generated_data/Concentration_data_high_res.npy")

shape_Conc = data_in_Conc.shape
print("Data shapes:", shape_Conc)

In [ ]:
#Adjust the data shapes for easier model evaluation
data_in_Conc_reshaped = np.reshape(data_in_Conc, (shape_Conc[0]*shape_Conc[1], shape_Conc[2], shape_Conc[3], 1))
data_target_Conc_reshaped = np.reshape(data_target_Conc, (shape_Conc[0]*shape_Conc[1], shape_Conc[2], shape_Conc[3], 1))

print("Data reshaped:", data_in_Conc_reshaped.shape)

### DAMN model

In [ ]:
#Use the DAMN model to predict high-resolution images
DAMN_model_output_Conc_reshaped = np.squeeze(DAMN_model.predict(data_in_Conc_reshaped))

#And return to the original shape
DAMN_model_output_Conc = DAMN_model_output_Conc_reshaped.reshape((shape_Conc[0], shape_Conc[1], shape_Conc[2], shape_Conc[3]))

np.save("Data_files/Evaluated_data/Concentration_DAMN_model_output.npy", DAMN_model_output_Conc)

In [ ]:
#Evaluate a chosen metric comparing the DAMN output with the corresponding target image
DAMN_model_errors_Conc = Evaluate_metric(DAMN_model_output_Conc, data_target_Conc)

np.save("Data_files/Evaluated_data/Concentration_DAMN_model_errors.npy", DAMN_model_errors_Conc)

### Richardson-Lucy algorithm

In [ ]:
#As the RL algorithm is very time consuming (matter of up to hours), you can choose to evaluate only a small portion of the data for the graph visualization
#Namely, 1/10 of data samples in every 4-th point on the horizontal axis
use_all_data_Conc = True             #Switch to False for evaluating only the reduced dataset 

if use_all_data_Conc:
    RL_data_in_Conc = data_in_Conc
    RL_data_target_Conc = data_target_Conc
    RL_output_Conc = np.zeros(data_in_Conc.shape)
else:
    RL_data_in_Conc = data_in_Conc[::4,:10]
    RL_data_target_Conc = data_target_Conc[::4,:10]
    RL_output_Conc = np.zeros(data_in_Conc[::4,:10].shape)

print("Using the following data shape for RL slgorithm:", RL_data_in_Conc.shape)

##### The following cell runs the Richardson-Lucy algorithm, which might turn very time consuming

In [ ]:
#The kernel stays the same for all Concentration data samples
kernel = Gauss_kernel(PSF_width_value)

#The ProcessPoolExecutor calling RL_iteration_for_concurrent function to evaluate the RL_data_in_Conc data
start = time.time()
for i in range(RL_data_in_Conc.shape[0]):
    start_i = time.time()
    with concurrent.futures.ProcessPoolExecutor(CPU_units_to_use) as pool:
        intermediate_func = functools.partial(RL_iteration_for_concurrent, kernel)
        res = pool.map(intermediate_func, RL_data_in_Conc[i])
    RL_output_Conc[i] = np.array(list(res))
    print("Finished iteration", i+1, "out of", RL_data_in_Conc.shape[0], "in", np.round(time.time()-start_i, 2), "seconds (" + str(np.round(time.time()-start, 2)), "from start).")

np.save("Data_files/Evaluated_data/Concentration_RL_output.npy", RL_output_Conc)

In [ ]:
#Evaluate a chosen metric comparing the DAMN output with the corresponding target image
RL_errors_Conc = Evaluate_metric(RL_output_Conc, RL_data_target_Conc)

np.save("Data_files/Evaluated_data/Concentration_RL_errors.npy", RL_errors_Conc)

## Evaluate dataset of the Airy-to-Gauss kernel transition graph - panel (D)

In [ ]:
#Loading the data generated in the Data_generation notebook
data_in_AtoG = np.load("Data_files/Generated_data/Transition_data_low_res.npy")
data_target_AtoG = np.load("Data_files/Generated_data/Transition_data_high_res.npy")
horizontal_axis_AtoG = np.load("Data_files/Generated_data/Transition_axis_array.npy")

shape_AtoG = data_in_AtoG.shape
print("Data shapes:", shape_AtoG)

In [ ]:
#Adjust the data shapes for easier model evaluation
data_in_AtoG_reshaped = np.reshape(data_in_AtoG, (shape_AtoG[0]*shape_AtoG[1], shape_AtoG[2], shape_AtoG[3], 1))
data_target_AtoG_reshaped = np.reshape(data_target_AtoG, (shape_AtoG[0]*shape_AtoG[1], shape_AtoG[2], shape_AtoG[3], 1))

print("Data reshaped:", data_in_AtoG_reshaped.shape)

### DAMN model

In [ ]:
#Use the DAMN model to predict high-resolution images
DAMN_model_output_AtoG_reshaped = np.squeeze(DAMN_model.predict(data_in_AtoG_reshaped))

#And return to the original shape
DAMN_model_output_AtoG = DAMN_model_output_AtoG_reshaped.reshape((shape_AtoG[0], shape_AtoG[1], shape_AtoG[2], shape_AtoG[3]))

np.save("Data_files/Evaluated_data/Transition_DAMN_model_output.npy", DAMN_model_output_AtoG)

In [ ]:
#Evaluate a chosen metric comparing the DAMN output with the corresponding target image
DAMN_model_errors_AtoG = Evaluate_metric(DAMN_model_output_AtoG, data_target_AtoG)

np.save("Data_files/Evaluated_data/Transition_DAMN_model_errors.npy", DAMN_model_errors_AtoG)

### Richardson-Lucy algorithm

In [ ]:
#As the RL algorithm is very time consuming (matter of up to hours), you can choose to evaluate only a small portion of the data for the graph visualization
#Namely, 1/10 of data samples in every 4-th point on the horizontal axis
use_all_data_AtoG = True             #Switch to False for evaluating only the reduced dataset 

if use_all_data_AtoG:
    RL_data_in_AtoG = data_in_AtoG
    RL_data_target_AtoG = data_target_AtoG
    RL_output_AtoG = np.zeros(data_in_AtoG.shape)
    RL_horizontal_axis_AtoG = horizontal_axis_AtoG
else:
    RL_data_in_AtoG = data_in_AtoG[::4,:10]
    RL_data_target_AtoG = data_target_AtoG[::4,:10]
    RL_output_AtoG = np.zeros(data_in_AtoG[::4,:10].shape)
    RL_horizontal_axis_AtoG = horizontal_axis_AtoG[::4]

print("Using the following data shape for RL slgorithm:", RL_data_in_AtoG.shape)

##### The following cell runs the Richardson-Lucy algorithm, which might turn very time consuming

In [ ]:
#The ProcessPoolExecutor calling RL_iteration_for_concurrent function to evaluate the RL_data_in_AtoG data
start = time.time()
for i in range(RL_data_in_AtoG.shape[0]):
    start_i = time.time()

    #The kernel stays the same only for data with the same horizontal axis position
    kernel = Get_AtoG_kernel(2, RL_horizontal_axis_AtoG[i])
    with concurrent.futures.ProcessPoolExecutor(CPU_units_to_use) as pool:
        intermediate_func = functools.partial(RL_iteration_for_concurrent, kernel)
        res = pool.map(intermediate_func, RL_data_in_AtoG[i])
    RL_output_AtoG[i] = np.array(list(res))
    print("Finished iteration", i+1, "out of", RL_data_in_AtoG.shape[0], "in", np.round(time.time()-start_i, 2), "seconds (" + str(np.round(time.time()-start, 2)), "from start).")

np.save("Data_files/Evaluated_data/Transition_RL_output.npy", RL_output_AtoG)

In [ ]:
#Evaluate a chosen metric comparing the DAMN output with the corresponding target image
RL_errors_AtoG = Evaluate_metric(RL_output_AtoG, RL_data_target_AtoG)

np.save("Data_files/Evaluated_data/Transition_RL_errors.npy", RL_errors_AtoG)